In [1]:
import xgboost as xgb
import numpy as np
import pandas as pd
import torch

In [2]:
X_train = pd.read_pickle('X_train.pkl')
y_train = pd.read_pickle('y_train.pkl')
X_test = pd.read_pickle('X_test.pkl')
y_test = pd.read_pickle('y_test.pkl')


In [3]:
# Converte os tipos das variáveis para boleano
X_train = X_train.astype(np.float32)
X_test = X_test.astype(np.float32)
y_train = y_train.astype(np.int8)
y_test = y_test.astype(np.int8)

In [4]:
X_train = X_train.to_numpy()
X_test = X_test.to_numpy()

y_train = y_train.to_numpy()
y_test = y_test.to_numpy()

In [5]:
X_train = torch.from_numpy(X_train).type(torch.float)
X_test = torch.from_numpy(X_test).type(torch.float)

y_train = torch.from_numpy(y_train).type(torch.LongTensor)
y_test = torch.from_numpy(y_test).type(torch.LongTensor)

y_train = y_train.squeeze()
y_test = y_test.squeeze()

print("FIM")

FIM


In [6]:
X_train.shape

torch.Size([900000, 741])

In [7]:
from torch import nn
    
class Model_Mul1_4_v1(nn.Module):
    def __init__(self):
        super().__init__()
        self.linear_layer_stack = nn.Sequential(
            nn.Linear(741, 4096),
            nn.LeakyReLU(inplace=True),
            nn.Dropout(p=0.25),
            nn.Linear(4096, 512),
            nn.LeakyReLU(inplace=True),
            nn.Dropout(p=0.25),
            nn.Linear(512, 4)
        )

    def forward(self, x):
        return self.linear_layer_stack(x)

In [8]:
# importando o ReduceLROnPlateau
from torch.optim.lr_scheduler import StepLR

# Criando uma instância do nosso modelo
model = Model_Mul1_4_v1()

total_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
print(f'{total_params:,} parâmetros livres')

# Cria uma função de erro otimizada para binário
loss_fn = nn.CrossEntropyLoss()

# Cria o otimizador do modelo
optimizer = torch.optim.Adam(model.parameters(),
                            lr=0.0004059707459502205,
                            weight_decay=8.8e-07,
                            betas=(0.9, 0.999),
                            eps=1e-08,
                            amsgrad=True
                            ) 

# Cria o escalonador de taxa de aprendizado
scheduler = StepLR(optimizer, step_size=1, gamma=0.997)

5,138,948 parâmetros livres


In [9]:
from sklearn.metrics import accuracy_score
from torch.utils.data import TensorDataset, DataLoader

epochs = 500 

batch_size = 1024

# Calcula o número de mini-batches com base no tamanho do conjunto de treinamento
num_batches = len(X_train) // batch_size

# Calcula o número de mini-batches com base no tamanho do conjunto de teste
num_test_batches = len(X_test) // batch_size

# Cria um DataLoader para carregar os dados de treinamento em mini-batches
train_data = TensorDataset(X_train, y_train)
train_loader = DataLoader(train_data, batch_size=batch_size, shuffle=True, num_workers=2)

# Cria um DataLoader para carregar os dados de teste em mini-batches
test_data = TensorDataset(X_test, y_test)
test_loader = DataLoader(test_data, batch_size=batch_size, shuffle=False, num_workers=2)

del X_train, y_train, X_test, y_test

# Cria listas para armazenar os valores de perda, acurácia e taxa de aprendizado
store_train_loss = []
store_test_loss = []
store_train_acc = []
store_test_acc = []
store_lr = []

# Loop das épocas
for epoch in range(epochs):

    ### Treinamento
    # Habilita o modelo para treinamento
    model.train()

    train_loss = 0.0
    train_acc = 0.0
    num_samples = 0

    num_train_batch = 1
    num_test_batch = 1

    # Loop dos mini-batches de treinamento
    for inputs, targets in train_loader:

        # Print atualizando a evolução do treinamento, como uma barra de progresso
        print(f'\rEpoch: {epoch} | Treinamento: {num_train_batch}/{(num_batches)}', end='')

        # Zera os gradientes
        optimizer.zero_grad()

        # Forward pass
        y_logits = model(inputs)
        y_pred = torch.softmax(y_logits, dim=1).argmax(dim=1)

        # Calcula a perda do treinamento
        loss = loss_fn(y_logits, targets)
        train_loss += loss.item()

        # Calcula a acurácia do treinamento
        batch_acc =  accuracy_score(targets.detach(), y_pred.detach())
        train_acc += batch_acc * len(inputs)
        num_samples += len(inputs)

        # Backward pass
        loss.backward()

        # Atualiza os pesos
        optimizer.step()

        num_train_batch += 1

    # Calcula a perda e acurácia média do treinamento
    train_loss /= num_batches
    train_acc /= num_samples

    # Armazena os valores de perda e acurácia do treinamento
    store_train_loss.append(train_loss)
    store_train_acc.append(train_acc)

    ### Validação
    # Desabilita o modelo para treinamento
    model.eval()

    val_loss = 0.0
    val_acc = 0.0
    val_samples = 0

    # Loop dos mini-batches de teste
    with torch.inference_mode():
        for inputs, targets in test_loader:

            # Print atualizando a evolução do teste, como uma barra de progresso
            print(f'\rEpoch: {epoch} | Treinamento: {num_train_batch}/{(num_batches)} | Teste: {num_test_batch}/{(num_test_batches)} | ', end='')

            # Forward pass
            test_logits = model(inputs)
            y_pred = torch.softmax(test_logits, dim=1).argmax(dim=1)

            loss = loss_fn(test_logits, targets)
            val_loss += loss.item()

            # Calcula a acurácia do teste
            batch_acc = accuracy_score(targets.detach(), y_pred.detach())
            val_acc += batch_acc * len(inputs)

            val_samples += len(inputs)

            num_test_batch += 1

    # Calcula a perda e acurácia média do teste
    val_loss /= num_test_batches
    val_acc /= val_samples

    # Armazena os valores de perda e acurácia do teste
    store_test_loss.append(val_loss)
    store_test_acc.append(val_acc)

    # Apresenta os resultados do treinamento e teste
    print(f'Loss: {train_loss:.4f}, Acc: {train_acc:.4f} | Test Loss: {val_loss:.4f}, Test acc: {val_acc:.4f} | Learning rate: {optimizer.param_groups[0]["lr"]:.8f}')

    # Atualiza o escalonador de taxa de aprendizado
    scheduler.step()

    # Armazena a taxa de aprendizado
    store_lr.append(optimizer.param_groups[0]["lr"])

    # Salva o modelo a cada 50 épocas
    if epoch % 50 == 0:
        torch.save(model.state_dict(), f'model_mul1-4_v4_{epoch}.pth')


Epoch: 0 | Treinamento: 880/878 | Teste: 118/117 | Loss: 1.1984, Acc: 0.4530 | Test Loss: 1.1171, Test acc: 0.5046 | Learning rate: 0.00040597
Epoch: 1 | Treinamento: 880/878 | Teste: 118/117 | Loss: 1.0725, Acc: 0.5226 | Test Loss: 1.0165, Test acc: 0.5516 | Learning rate: 0.00040475
Epoch: 2 | Treinamento: 880/878 | Teste: 118/117 | Loss: 1.0183, Acc: 0.5490 | Test Loss: 0.9881, Test acc: 0.5683 | Learning rate: 0.00040354
Epoch: 3 | Treinamento: 880/878 | Teste: 118/117 | Loss: 0.9826, Acc: 0.5669 | Test Loss: 0.9512, Test acc: 0.5849 | Learning rate: 0.00040233
Epoch: 4 | Treinamento: 880/878 | Teste: 118/117 | Loss: 0.9556, Acc: 0.5807 | Test Loss: 0.9216, Test acc: 0.5992 | Learning rate: 0.00040112
Epoch: 5 | Treinamento: 880/878 | Teste: 118/117 | Loss: 0.9313, Acc: 0.5927 | Test Loss: 0.9248, Test acc: 0.5981 | Learning rate: 0.00039992
Epoch: 6 | Treinamento: 880/878 | Teste: 118/117 | Loss: 0.9112, Acc: 0.6036 | Test Loss: 0.8848, Test acc: 0.6214 | Learning rate: 0.00039872

KeyboardInterrupt: 